 ### Introduction What is the project goal? <b> </br>
 Goal: To build algorithms to rate the complexity of reading passages for grade 3-12 classroom use.

In [ ]:
import os
import gc
import sys
import cv2
import math
import time
import tqdm
import random
import numpy as np
import pandas as pd
import seaborn as sns
from tqdm import tqdm
import matplotlib.pyplot as plt

import warnings
warnings.filterwarnings('ignore')

import optuna
import lightgbm as lgb
import xgboost as xgb
from sklearn.svm import SVR
from catboost import CatBoostRegressor, Pool, CatBoost

from sklearn.metrics import mean_squared_error
from sklearn.model_selection import KFold,StratifiedKFold

import torch
import torchvision
import torch.nn as nn
import torch.optim as optim
from torch.nn import Parameter
import torch.nn.functional as F
from torch.optim import Adam, lr_scheduler
from torch.utils.data import Dataset, DataLoader
from torch.optim.optimizer import Optimizer
from torch.optim.lr_scheduler import _LRScheduler
from torch.optim.lr_scheduler import (CosineAnnealingWarmRestarts, CosineAnnealingLR, 
                                      ReduceLROnPlateau)

from transformers import (AutoModel, AutoTokenizer, 
                          AutoModelForSequenceClassification,get_constant_schedule_with_warmup)

import plotly.express as px
import plotly.graph_objs as go
import plotly.figure_factory as ff


from colorama import Fore, Back, Style
y_ = Fore.YELLOW
r_ = Fore.RED
g_ = Fore.GREEN
b_ = Fore.BLUE
m_ = Fore.MAGENTA
c_ = Fore.CYAN
sr_ = Style.RESET_ALL
class color:
    BOLD = '\033[1m' + '\033[93m'
    END = '\033[0m'

#### Get the data
We are going to read the train data here . The columns listed as in the data description of the competition are 
* id - unique ID for excerpt
* url_legal - URL of source - this is blank in the test set.
* license - license of source material - this is blank in the test set.
* excerpt - text to predict reading ease of
* target - reading ease
* standard_error - measure of spread of scores among multiple raters for each excerpt. Not included for test data.

In [ ]:
train = pd.read_csv("../input/commonlitreadabilityprize/train.csv")
test = pd.read_csv("../input/commonlitreadabilityprize/test.csv")

print(color.BOLD + "Train Shape:" + color.END, train.shape, "\n" +
      color.BOLD + "Test Shape:" + color.END, test.shape)

train.isna().sum()

In [ ]:
train.describe()

As shown above we dont have any missing any values in the columns of our concern which is excerpt and target

#### EDA
##### Lets plot the distribution of target and standard error 

In [ ]:

from matplotlib import pyplot
pyplot.hist(train['target'], bins=50,  label='target_bins')
pyplot.hist(train['standard_error'], bins=100, label='error_bins')
pyplot.legend(loc='upper right')
pyplot.show()

Lets understand how the standard error,target and excerpt are related. We are going to look at few examples where target is high meaning excerpt is difficult.

In [ ]:
pd.set_option('display.max_colwidth', 10000)
train.sort_values('target', ascending=False).head(3)[['excerpt','standard_error','target']]

In [ ]:
pd.set_option('display.max_colwidth', 10000)
train.sort_values('target', ascending=True).head(3)[['excerpt','standard_error','target']]

In [ ]:
train[(train['target']<=-0.94)&
      (train['target']>=-0.95)].head(3)[['excerpt','standard_error','target']]

In [ ]:
## Lets find the correlation between target and standard error
from scipy.stats import pearsonr
corr, _=pearsonr(train["target"], train["standard_error"])
print(corr)


Although technically a negative correlation, the relationship between our variables is only weak (nb. the nearer the value is to zero, the weaker the relationship.<b> There is no correlation between standard error and target.

In [ ]:
sns.scatterplot(x=train["target"], y=train["standard_error"], 
                size=train["standard_error"], sizes=(20, 100))

As seen from the graph the standard error reduces towards the middle, but increases towards both ends. This means regardless of the excerpt being complex or simple people rating vary widely and thus standard error rises at both ends

Lets have a look at whats the impact of having licence and url_legal for an excerpt on that score

In [ ]:
train_licence=train[~(train.license.isnull())]
print(train_licence.shape)

In [ ]:
corr, _=pearsonr(train_licence["target"], train_licence["standard_error"])
print(corr)

In [ ]:
sns.scatterplot(x=train_licence["target"], y=train_licence["standard_error"], 
                size=train["standard_error"], sizes=(20, 100))


There is no impact in correlation even if the excerpt has a licence and url_legal

### Baseline Model code to follow